In [97]:
import pandas as pd
import numpy as np
import os
import sys

### Read the data

In [98]:
current_path = os.getcwd()
day = int(current_path.split('day')[1])

fn = 'day' + str(day) + '.txt'

file_content = open(fn).read().split('\n')

test_fn = 'day' + str(day) + 'test.txt'

test_file_content = open(test_fn).read().split('\n')

### Part 1

In [99]:
def get_next_positions(current_position, maze, part=1):
    y, x = current_position
    next_positions = []
    if x < len(maze[y]) - 1:
        if part == 1 and maze[y][x + 1] in ['.', '>'] and maze[y][x] not in ['<', 'v', '^']:
            next_positions.append((y, x + 1))
        elif part == 2 and maze[y][x + 1] in ['.', '>', 'v', '^', '<']:
            next_positions.append((y, x + 1))
    if x > 0:
        if part == 1 and maze[y][x - 1] in ['.', '<'] and maze[y][x] not in ['>', 'v', '^']:
            next_positions.append((y, x - 1))
        elif part == 2 and maze[y][x - 1] in ['.', '>', 'v', '^', '<']:
            next_positions.append((y, x - 1))
    if y > 0:
        if part == 1 and maze[y - 1][x] in ['.', '^'] and maze[y][x] not in ['<', 'v', '>']:
            next_positions.append((y - 1, x))
        elif part == 2 and maze[y - 1][x] in ['.', '>', 'v', '^', '<']:
            next_positions.append((y - 1, x))
    if y < len(maze) - 1:
        if part == 1 and maze[y + 1][x] in ['.', 'v'] and maze[y][x] not in ['<', '>', '^']:
            next_positions.append((y + 1, x))
        elif part == 2 and maze[y + 1][x] in ['.', '>', 'v', '^', '<']:
            next_positions.append((y + 1, x))
    return next_positions



def find_longest_path(current_position, maze, path, part=1):
    next_positions = get_next_positions(current_position, maze, part)
    if len(next_positions) == 0:
        return path

    longest_new_path = path
    length_of_longest_new_path = len(path)
    for position in next_positions:
        if position not in path:
            new_path = find_longest_path(position, maze, path + [position], part)
            if len(new_path) > length_of_longest_new_path:
                longest_new_path = new_path
                length_of_longest_new_path = len(new_path)

    return longest_new_path


def display_path(maze, path):
    print(len(list(set(path))))
    for y, row in enumerate(maze):
        for x, char in enumerate(row):
            if (y, x) in path:
                print('O', end='')
            else:
                print(char, end='')
        print('')

In [100]:

working_file_content = test_file_content
start = (0, 1) # (y, x)
print(len(find_longest_path(start, working_file_content, [])))

94


### Part 2

In [103]:
data = file_content

# Part 1
edges = {}
for r, row in enumerate(data):
    for c, v in enumerate(row):
        if v == ".":
            for dr, dc in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
                ar, ac = r + dr, c + dc
                if not (0 <= ar < len(data) and 0 <= ac < len(row)):
                    continue
                if data[ar][ac] == ".":
                    edges.setdefault((r, c), set()).add((ar, ac))
                    edges.setdefault((ar, ac), set()).add((r, c))
        if v == ">":
            edges.setdefault((r, c), set()).add((r, c + 1))
            edges.setdefault((r, c - 1), set()).add((r, c))
        if v == "v":
            edges.setdefault((r, c), set()).add((r + 1, c))
            edges.setdefault((r - 1, c), set()).add((r, c))

n, m = len(data), len(data[0])

q = [(0, 1, 0)]
visited = set()
best = 0
while q:
    r, c, d = q.pop()
    if d == -1:
        visited.remove((r, c))
        continue
    if (r, c) == (n - 1, m - 2):
        best = max(best, d)
        continue
    if (r, c) in visited:
        continue
    visited.add((r, c))
    q.append((r, c, -1))
    for ar, ac in edges[(r, c)]:
        q.append((ar, ac, d + 1))
print(best)

# Part 2
edges = {}  # (r, c) -> (ar, ac, length)
for r, row in enumerate(data):
    for c, v in enumerate(row):
        if v in ".>v":
            for dr, dc in [(-1, 0), (0, -1), (0, 1), (1, 0)]:
                ar, ac = r + dr, c + dc
                if not (0 <= ar < len(data) and 0 <= ac < len(row)):
                    continue
                if data[ar][ac] in ".>v":
                    edges.setdefault((r, c), set()).add((ar, ac, 1))
                    edges.setdefault((ar, ac), set()).add((r, c, 1))

# Remove nodes with degree 2 by merging the edges
while True:
    for n, e in edges.items():
        if len(e) == 2:
            a, b = e
            edges[a[:2]].remove(n + (a[2],))
            edges[b[:2]].remove(n + (b[2],))
            edges[a[:2]].add((b[0], b[1], a[2] + b[2]))
            edges[b[:2]].add((a[0], a[1], a[2] + b[2]))
            del edges[n]
            break
    else:
        break

n, m = len(data), len(data[0])

q = [(0, 1, 0)]
visited = set()
best = 0
while q:
    r, c, d = q.pop()
    if d == -1:
        visited.remove((r, c))
        continue
    if (r, c) == (n - 1, m - 2):
        best = max(best, d)
        continue
    if (r, c) in visited:
        continue
    visited.add((r, c))
    q.append((r, c, -1))
    for ar, ac, l in edges[(r, c)]:
        q.append((ar, ac, d + l))
print(best)

2206
6490
